In [14]:
import pandas as pd
import re
import PyPDF2

import tabula as tb

In [15]:
def scrape_pdf(file):
    template_path = 'pdf_template.tabula-template.json'
    return tb.read_pdf_with_template(file, template_path, stream=True)
    

In [21]:
pdf_path = "2015_house_pdfs/Pelosi_Nancy_10010857.pdf"
db = scrape_pdf(pdf_path)

In [18]:
pdf_path_joe = "2015_house_pdfs/Wilson_Joe_10012512.pdf"

wrong_pdf_format_db = scrape_pdf(pdf_path_joe)

Error from tabula-java:
Exception in thread "main" java.lang.IndexOutOfBoundsException: Page number does not exist.
	at technology.tabula.ObjectExtractor.extractPage(ObjectExtractor.java:19)
	at technology.tabula.PageIterator.next(PageIterator.java:30)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:161)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




CalledProcessError: Command '['java', '-Dfile.encoding=UTF8', '-jar', '/home/kira/.local/lib/python3.10/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', '5', '--area', '66.172,17.212,737.842,561.893', '--stream', '--format', 'JSON', '2015_house_pdfs/Wilson_Joe_10012512.pdf']' returned non-zero exit status 1.

In [22]:
file = open(pdf_path, 'rb')


readpdf = PyPDF2.PdfFileReader(file)
totalpages = readpdf.numPages
print(totalpages)

9
